In [ ]:
"""
Original @author: Jessie - 2018-10-03

Updated by @author: Jade - 2018-11-27

"""
# imports 
%matplotlib inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from IPython.display import Image
import sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report 
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# load data
#import numpy array
def LoadSplitData(split):
    DATA_DIR = "Data"
    ENCODING_DIR = os.path.join(DATA_DIR, "binEncoding.csv")
   
    data_xy = pd.read_csv(ENCODING_DIR)
    data_xy = data_xy.drop(columns= ['Unnamed: 0'])

    #drop the target variables you do not need (1-7), market moving or not ratings
    colsxy= [0,1,2,4,5,6] #needs change
    data_xy = data_xy.drop(data_xy.columns[colsxy],axis=1)
    
    cols =[0,1,2,3,4,5,6]
    X = data_xy.drop(data_xy.columns[cols],axis=1)
    y = data_xy.iloc[:,3] #needs change
    
    
    #Get dummy test/train set 
    DummyX_train, DummyX_test, Dummyy_train, Dummyy_test = train_test_split(data_xy, y, test_size=split, random_state=42)
    
    return X, y, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test, data_xy

In [ ]:
X, y, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test, data_xy = LoadSplitData(0.30) #Decide the test size (% of total data)

In [ ]:
#Use Linear Regression - Testing with dummy-y-variable
def SingleTest(X_train, y_train, X_test, y_test):
    #define classifier
    linReg = LinearRegression()

    #Test 
    clfSingleTest = linReg.fit(X_train, y_train)

    # predict on train and test set
    y_train_predict = clfSingleTest.predict(X_train)
    y_test_predict = clfSingleTest.predict(X_test)

    #Binary metrics
    columns = ['Precision','Recall', 'F1', 'Avg Precision', 'Accuracy']
    df = pd.DataFrame(index=['Train','Test'], columns=columns)
    
    
    TrainPrecision = precision_score(y_train, y_train_predict)
    TestPrecision = precision_score(y_test, y_test_predict)
    
    TrainRecall = accuracy_score(y_train, y_train_predict)
    TestRecall = accuracy_score(y_test, y_test_predict)
    
    Trainf1 = f1_score(y_train, y_train_predict, average='binary')
    Testf1 = f1_score(y_test, y_test_predict, average='binary')
    
    #Not to be confused with the ranking metric, mAP (mean average precision), this is simply the average of the P and R curve
    TrainAvgP = average_precision_score(y_train, y_train_predict)
    TestAvgP = average_precision_score(y_test, y_test_predict)
    
    TrainAccuracy = accuracy_score(y_train, y_train_predict)
    TestAccuracy = accuracy_score(y_test, y_test_predict)
    
    df.loc['Train'] = pd.Series({'Precision': TrainPrecision, 'Recall': TrainRecall, 'F1': Trainf1, 'Avg Precision': TrainAvgP, 'Accuracy': TrainAccuracy})
    df.loc['Test'] = pd.Series({'Precision': TestPrecision, 'Recall': TestRecall, 'F1': Testf1, 'Avg Precision': TestAvgP, 'Accuracy': TestAccuracy})
    return df

In [ ]:
def SequentialSetRun(X, y, testsize):
   
    #Predicting on Real DataSet - Only 1 run
    num_articles = len(X) -1 #Subtract header row
    #testsize = 0.30
    trainsize = 1-testsize


    #Select first 70% as train
    X_train = X.iloc[:round(num_articles*trainsize)]
    y_train = y.iloc[:round(num_articles*trainsize)]

    #Followiing 30% is test
    X_test = X.iloc[(round(num_articles*trainsize)):]
    y_test = y.iloc[(round(num_articles*trainsize)):]
   
    #Run SingleTest
    TestResults = SingleTest(X_train, y_train, X_test, y_test)

    return TestResults

In [ ]:
# create variables for experiment
X, y, DummyX_train, DummyX_test, Dummyy_train, Dummyy_test, data_xy = LoadSplitData(0.30) #Decide the test size (% of total data)

In [ ]:
#Sanity check with dummy variable:  output should be 1.0)
df = SingleTest(DummyX_train, Dummyy_train, DummyX_test, Dummyy_test)
df.head()

In [ ]:
# sequentially split data and run LR
dfSeqTest = SequentialSetRun(X, y, 0.30) #final number is % size of testing set
dfSeqTest.head()